In [4]:
import pandas as pd
data = pd.read_csv('owid-covid-data.csv')

- DATA_LIST
    - USA: 미국
    - NAM: 북아메리카
    - KOR: 한국
    - SAM: 남아메리카
    - JPN: 일본
    - CHN: 중국
    - RSI: 러시아
    - EEU: 동유럽
    - WEU: 서유럽
    - MAS: 중동
    - AUS: 오세아니아
    - AFR: 아프리카
    - SAF: 남아공 
    - HKO: 홍콩
    - ASIA: 아시아
        - 중동,한국,일본,중국,홍콩을 제외한 나라는 아시아로 통합하여 분류하였고 
        - 중국에 속하지만 마카오는 아시아로 분류함

In [5]:
DATA_LIST=['USA','NAM','KOR','SAM','JPN','CHN','RSI','EEU','WEU','MAS','AUS','AFR','SAF','HKO','ASIA']
len(DATA_LIST)

15

In [12]:
C_L_dict={'Africa':{'SAF':['South Africa'],'AFR':[]},
          'Asia':{'KOR':['South Korea'],'JPN':['Japan'],
                  'MAS':['Afghanistan','Bahrain','Kuwait','Saudi Arabia'],
                  'CHN':['China'],
                  'HKO':['Hong Kong'],
                  'ASIA':[]},
          'Europe':{'RSI':['Russia'],
                    'WEU':['Andorra','Austria','Belgium','England',
                           'France','Guernsey','Iceland','Ireland','Isle of Man',
                           'Italy','Jersey','Luxembourg','Netherlands','Poland','Portugal',
                           'Scotland','Spain','Sweden','United Kingdom','Wales'],
                    'EEU':[]},
          'North America':{'USA':['United States'],'NAM':[]},
          'Oceania':{'AUS':[]},
          'South America':{'SAM':[]}}
DATA_LIST=['USA','NAM','KOR','SAM','JPN','CHN','RSI','EEU','WEU','MAS','AUS','AFR','SAF','HKO','ASIA']
data_dict={i:pd.DataFrame(columns=["date", "loc", "total_cases",'total_deaths','people_vaccinated']) for i in DATA_LIST}

In [10]:
def segment_by_continent_and_location(group,continent,location,data_dict,C_L_dict):
    """
        데이터 분류의 효율성을 높이기 위해 대륙 별로 분류된 데이터에서
        우리가 원하는 지역별 데이터를 분류하여
        data_dict에 담는다.
    """
    continents,location_dict=C_L_dict.keys(),C_L_dict.values()
    for continent_ in continents:
        if continent != continent_:
            continue
        for name,loction_list in C_L_dict[continent_].items():
            if loction_list:
                if location in loction_list:
                    data_dict[name] = pd.concat([data_dict.get(name, pd.DataFrame({})), group])
            else:
                data_dict[name] = pd.concat([data_dict.get(name, pd.DataFrame({})), group])

In [ ]:
pd.concat([])

In [11]:
# pd.to_datetime(): datetime(날짜와 시간) 객체로 변환.
# dt.: datetime 구성 요소(예: 연, 월, 일 등)에 액세스.
# to_period('M'): 월별로 그룹화.
data['month'] = pd.to_datetime(data['date']).dt.to_period('M') 
for month, month_group in data.groupby('month'):
    data_segment_by_month_dict= {i: pd.DataFrame({}) for i in DATA_LIST}

    for continent, continent_group in month_group.groupby('continent'):
        for location, location_group in continent_group.groupby('location'):
            segment_by_continent_and_location(location_group,continent,location,data_segment_by_month_dict,C_L_dict)

    for loc_code, loc_df in data_segment_by_month_dict.items():
        try:
            data_dict[loc_code] = pd.concat([data_dict.get(loc_code, pd.DataFrame({})),
                                       pd.DataFrame({'date': [month], 
                                                     'loc': [loc_code],
                                                     'total_cases': [loc_df['new_cases'].sum()],
                                                     'total_deaths': [loc_df['new_deaths'].sum()],
                                                     'total_vaccinations': [loc_df['new_vaccinations'].sum()]})])
            filename = f'country/{loc_code}.csv'
            data_dict[loc_code].to_csv(filename, index=False)
        except:
            pass
        
data_dict['KOR']

,date,loc,total_cases,total_deaths,people_vaccinated,total_vaccinations
0,2020-01,KOR,11.0,0.0,NaN,0.0
0,2020-02,KOR,2920.0,16.0,NaN,0.0
0,2020-03,KOR,6855.0,146.0,NaN,0.0
0,2020-04,KOR,979.0,85.0,NaN,0.0
0,2020-05,KOR,703.0,23.0,NaN,0.0
...,...,...,...,...,...,...
0,2023-07,KOR,1070190.0,199.0,NaN,0.0
0,2023-08,KOR,1234746.0,596.0,NaN,0.0
0,2023-09,KOR,135331.0,122.0,NaN,0.0
0,2023-10,KOR,0.0,0.0,NaN,0.0
